<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/Merton_KMV_py2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install --upgrade yfinance
import yfinance as yf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
K_0=202468383000
D_0=216339988000

In [31]:
# import yfinance as yf

symbol = 'WALMEX.MX'

# Crear un objeto Ticker con el símbolo de la empresa
ticker = yf.Ticker(symbol)

# Obtener la información financiera más reciente
info = ticker.info

# Obtener los valores de interés
Acc = info.get('sharesOutstanding')
Kp = info.get('enterpriseValue')
Dp = info.get('totalDebt')

# Imprimir los resultados
print(Kp, "vs", K_0)
print("Valor empresa entre valor contable:", Kp / K_0)
print(Dp, "vs", D_0)
print("Valor deuda entre pasivos totales:", Dp / D_0)

1282728525824 vs 168144767000.0
Valor empresa entre valor contable: 7.628715116801702
70001532928 vs 113498948000.0
Valor deuda entre pasivos totales: 0.6167593106501745


In [34]:
import pandas as pd
import numpy as np
from statistics import stdev

df = pd.DataFrame()


hist=ticker.history(period="max", auto_adjust=True)
print(hist.head(3))

df['ds'] = hist.index
df['y'] = hist['Close'].values

df.tail()

                               Open      High       Low     Close    Volume  \
Date                                                                          
2003-01-01 00:00:00-06:00  4.241726  4.406275  4.241726  4.356911         0   
2003-01-02 00:00:00-06:00  4.241726  4.406275  4.241726  4.356911   8334330   
2003-01-03 00:00:00-06:00  4.351425  4.378850  4.334970  4.351425  18475096   

                           Dividends  Stock Splits  
Date                                                
2003-01-01 00:00:00-06:00        0.0           0.0  
2003-01-02 00:00:00-06:00        0.0           0.0  
2003-01-03 00:00:00-06:00        0.0           0.0  


,ds,y
5151,2023-05-16 00:00:00-06:00,72.779999
5152,2023-05-17 00:00:00-06:00,72.279999
5153,2023-05-18 00:00:00-06:00,72.010002
5154,2023-05-19 00:00:00-06:00,72.169998
5155,2023-05-22 00:00:00-06:00,70.610001


In [35]:
Delta_A = df[df.ds<="2023-05-22"]['y'].pct_change(1) # 1 for ONE DAY lookback
vol_A=stdev(Delta_A[Delta_A.notnull()])*np.sqrt(252)

vol_A

0.2635114887400905

In [39]:
df[df.ds=="2023-05-22"]['y']*Acc
# print(Acc)

5155    1.232949e+12
Name: y, dtype: float64

In [44]:
A_0=df[df.ds=="2023-05-22"]['y']
A_0= A_0.values[0]*Acc
print(A_0)


print("Valor teórico de acciones entre valor de empresa yahoo",A_0/Kp)

print(K_0)
print("Valor teórico de acciones entre valor de capital social contable",A_0/np.array(K_0))
# print(K/A)

r=.113
T=1


1232949433024.3125
Valor teórico de acciones entre valor de empresa yahoo 0.9611928075212093
202468383000
Valor teórico de acciones entre valor de capital social contable 6.089589963408324


In [47]:
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np
x=np.array([D_0,vol_A])

K_0=K_0

A_0=A_0
# *(1-vol_A)

#Función para estimar vol_k y D
def G(x):
  vol_k = x[1]
  D = x[0]
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) 
  return F


ss=fsolve(G,x,full_output=True)

print(ss)

# !pip install gekko

# from gekko import GEKKO
# m=GEKKO()
# D,vol_k=[m.Var(1) for i in range(2)]
# m.Equations([A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)==0, A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) ==0 ])

# m.solve(disp=False)
# print(D.value,vol_k.value)


print(G(ss[0]))



D=ss[0][0]
vol_k=ss[0][1]

print("Valor de D", D)
print("Valor de vol_k", vol_k)

# print(D/D_0)
# print(vol_A/vol_K)


# #K_0=164370

# ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

# D=ss$root[1]
# vol_k=ss$root[2]
# ss$root[1]/D_0
# vol_A/ss$root[2]
# G(c(D,vol_k))
#Probabilidad de incumplimiento
PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

(array([5.57342619e+10, 2.53285348e-01]), {'nfev': 5, 'fjac': array([[-1.00000000e+00, -1.88642137e-10],
       [ 1.88642137e-10, -1.00000000e+00]]), 'r': array([-8.93150660e-01, -2.71014771e+02,  1.28272853e+12]), 'qtf': array([632.58208466,  27.28607166]), 'fvec': array([ 0.00000000e+00, -6.10351562e-05])}, 1, 'The solution converged.')
[ 0.00000000e+00 -6.10351562e-05]
Valor de D 55734261891.74116
Valor de vol_k 0.25328534767618116
0.0


Si la ecuación no converge es porque $A_0>K_0$

In [ ]:
D=ss[0][0]
vol_k=ss[0][1]

PD_merton=1-norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)))
print(PD_merton)

3.431250505947503e-09


## Conclusiones

1. Verificar el valor de $A_0$, que no sea mucho mayor que $K_0$.
2. Verificar el valor de la deuda.

# KMV


En KMV se encuentra el valor de $K_0$ y $\sigma_K$, dados los demás parámetros.

In [ ]:
print("A_0:", A_0)
print("Valor de K_0", K_0)
print("Valor de vol_A", vol_A)
print("Valor de Deuda Actual", D_0)
print("Tasa libre de riesgo", r)
print("Temporalidad", T)

A_0: 748430637344.3221
Valor de K_0 1267894190080
Valor de vol_A 0.2644493138143713
Valor de Deuda Actual 208507468000.0
Tasa libre de riesgo 0.05
Temporalidad 1


In [ ]:
K_0=Kp

x=np.array([K_0,vol_A])

D=D_0*4
#A_0=A_0*1/2

#Función para estimar vol_k y K_0
def G_kmv(x):
  vol_k = x[1]
  K_0 = x[0]  #modificar el valor inicial de K
  F=np.zeros(2)
  F[0]=A_0-K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)+D*np.exp(-r*T)*norm.cdf((np.log(K_0/D)+(r-0.5*vol_k**2)*T)/(vol_k*np.sqrt(T)),0,1)
  F[1]=A_0*vol_A-vol_k*K_0*norm.cdf((np.log(K_0/D)+(r+0.5*vol_k**2)*T)/(vol_k*np.sqrt(T))) 
  return F


ss=fsolve(G_kmv,x,full_output=True)

print(ss)





K_e=ss[0][0]
vol_k=ss[0][1]
print("Cociente K_0/D_0", K_e/D_0)
print("Cociente vol_K/vol_A",vol_k/vol_A)
print("Valor de solución en función:")
G_kmv(ss[0])

print("Deuda",D)
print("Valor empresa estimada",K_e)
print("Valor vol_k",vol_k)


DI=(K_e-D)/(vol_k*K_e)
print("DI")
DI
EDF=norm.cdf(-DI,0,1)
print("EDF Normal")
EDF


(array([1.54178439e+12, 1.28372025e-01]), {'nfev': 10, 'fjac': array([[-0.9982574 , -0.05900985],
       [ 0.05900985, -0.9982574 ]]), 'r': array([1.02218804e+00, 9.82578485e+10, 1.50833272e+12]), 'qtf': array([  79.5539131 , -336.55786685]), 'fvec': array([ 0.00000000e+00, -3.05175781e-05])}, 1, 'The solution converged.')
Cociente K_0/D_0 7.394384500387208
Cociente vol_K/vol_A 0.48543149255760965
Valor de solución en función:
Deuda 834029872000.0
Valor empresa estimada 1541784389594.1816
Valor vol_k 0.12837202511074597
DI
EDF Normal


0.00017449468649001438

## Conclusiones

Hace más sentido utilizar KMV cuando $A_0>K_0$, y tiene la ventaja de poder utilizar una deuda "objetivo", para calcular una probabilidad de incumplimiento.